In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('train.csv')

In [4]:
df.shape

(404290, 6)

In [5]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [6]:
df.isnull().sum()

id              0
qid1            0
qid2            0
question1       1
question2       2
is_duplicate    0
dtype: int64

In [10]:
df.dropna(subset=['question1','question2'], inplace=True)

In [11]:
df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [12]:
new_df = df

In [13]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [14]:
new_df.duplicated().sum()

0

In [15]:
ques_df = new_df[['question1','question2']]
ques_df.head()

,question1,question2
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?


In [17]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=30000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

MemoryError: Unable to allocate 181. GiB for an array with shape (808574, 30000) and data type int64

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=30000)
sparse_matrix = cv.fit_transform(questions)

q1_arr, q2_arr = np.vsplit(sparse_matrix, 2)


AxisError: axis1: axis 0 is out of bounds for array of dimension 0

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import vstack

questions = list(ques_df['question1']) + list(ques_df['question2'])

cv = CountVectorizer(max_features=30000)
sparse_matrix = cv.fit_transform(questions)

mid = sparse_matrix.shape[0] // 2
q1_sparse, q2_sparse = vstack(sparse_matrix[:mid]), vstack(sparse_matrix[mid:])


In [21]:
temp_df1 = pd.DataFrame(q1_sparse, index= ques_df.index)
temp_df2 = pd.DataFrame(q2_sparse, index= ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis=1)
temp_df.shape

(404287, 2)

In [22]:
temp_df

,0,0
0,"(0, 29297)\t1\n (0, 14271)\t1\n (0, 26886)...","(0, 29297)\t1\n (0, 14271)\t1\n (0, 26886)..."
1,"(0, 29297)\t1\n (0, 14271)\t1\n (0, 26886)...","(0, 29297)\t1\n (0, 26886)\t2\n (0, 15139)..."
2,"(0, 26886)\t1\n (0, 18953)\t1\n (0, 12903)...","(0, 4583)\t1\n (0, 12903)\t1\n (0, 4734)\t..."
3,"(0, 12903)\t1\n (0, 4734)\t1\n (0, 29368)\...","(0, 14271)\t1\n (0, 26886)\t1\n (0, 4583)\..."
4,"(0, 13472)\t1\n (0, 29324)\t1\n (0, 19053)...","(0, 13472)\t1\n (0, 29324)\t1\n (0, 29114)..."
...,...,...
404285,"(0, 26886)\t2\n (0, 13472)\t1\n (0, 18953)...","(0, 26886)\t1\n (0, 13472)\t2\n (0, 12903)..."
404286,"(0, 14271)\t1\n (0, 8455)\t1\n (0, 29839)\...","(0, 14271)\t2\n (0, 14341)\t1\n (0, 26881)..."
404287,"(0, 29297)\t1\n (0, 14271)\t1\n (0, 19053)...","(0, 29297)\t1\n (0, 26980)\t1\n (0, 5907)\t1"
404288,"(0, 29297)\t1\n (0, 14271)\t1\n (0, 26886)...","(0, 27181)\t1\n (0, 18953)\t1\n (0, 1895)\..."


In [23]:
temp_df['is_duplicate'] = new_df['is_duplicate']

In [24]:
temp_df.head()

,0,0,is_duplicate
0,"(0, 29297)\t1\n (0, 14271)\t1\n (0, 26886)...","(0, 29297)\t1\n (0, 14271)\t1\n (0, 26886)...",0
1,"(0, 29297)\t1\n (0, 14271)\t1\n (0, 26886)...","(0, 29297)\t1\n (0, 26886)\t2\n (0, 15139)...",0
2,"(0, 26886)\t1\n (0, 18953)\t1\n (0, 12903)...","(0, 4583)\t1\n (0, 12903)\t1\n (0, 4734)\t...",0
3,"(0, 12903)\t1\n (0, 4734)\t1\n (0, 29368)\...","(0, 14271)\t1\n (0, 26886)\t1\n (0, 4583)\...",0
4,"(0, 13472)\t1\n (0, 29324)\t1\n (0, 19053)...","(0, 13472)\t1\n (0, 29324)\t1\n (0, 29114)...",0


In [25]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values,test_size=0.2,random_state=1)

In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

ValueError: setting an array element with a sequence.

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)